In [57]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image

# Define the hyperparameters
batch_size = 64
lr = 0.0002
beta1 = 0.5
beta2 = 0.999
nz = 100
ngf = 64
ndf = 64
nc = 5
num_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the generator
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

class Discriminator(nn.Module):
    def __init__(self, nc, ndf):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)



In [48]:
def initialize_weights (model):
    for m in model.modules():
        if isinstance(m,(nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, .02)

In [58]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn import metrics
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim as optim
import math
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold

In [59]:
import glob

path = r"/home/jupyter/src/Thesis_Project/Data/50"
all_files = glob.glob(path + "/*")
print("Total number of subjects:", len(all_files))
print(np.load(all_files[0]).shape)

Total number of subjects: 75
(110, 5, 42, 42)


In [60]:
total = 40000
def extract_samples(target):
    positive_data = None
    negative_data = None

    positive_init = False
    negative_init = False

    positive_indices = None
    negative_indices = None

    print('**********************', target)
    for i, filename in enumerate(all_files):
        if i == target:
            positive_dataset = np.load(filename)
            positive_indices = list(range(len(positive_dataset)))
            np.random.shuffle(positive_indices)
            positive_init = True
            positive_data = positive_dataset[positive_indices]
            print("Current positive keystroke images Data shape is",positive_data.shape)

    return positive_data

In [61]:
class KeystrokesDataset(Dataset):
    def __init__(self, samples, labels, transform):
        self.samples = samples
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        x = self.samples[idx]
        x = x.transpose((1, 2 ,0))
        x = self.transform(image=x)['image']
        label = torch.from_numpy(self.labels[idx]).float()
        return x, label


In [62]:
def encapsulate(positive_data, negative_data):

    train_dataloaders = []
    test_dataloaders = []
  
    # try and use Cross Fold Val
    skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)

    total_dataset = np.concatenate((positive_data, negative_data), axis = 0)
    total_labels = np.concatenate((
      np.ones((len(positive_data), 1)), 
      np.zeros((len(negative_data), 1))
    ), axis=0)

    for train_index, test_index in skf.split(total_dataset, total_labels):
        train_dataset_numpy, test_dataset_numpy = total_dataset[train_index], total_dataset[test_index]
        train_labels_numpy, test_labels_numpy = total_labels[train_index], total_labels[test_index]

        mean = 0.
        std = 0.

        batch_samples = len(train_dataset_numpy) # batch size (the last batch can have smaller size!)
        images = train_dataset_numpy.reshape(batch_samples, 5, -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)

        mean /= len(train_dataset_numpy)
        std /= len(train_dataset_numpy)

    # transform = transforms.Compose([
    #   transforms.Normalize(mean=mean,std=std)
    # ])

    train_transform = A.Compose([
        A.Normalize(mean=mean.tolist(), std=std.tolist(), max_pixel_value=1.0, p=1.0),
        A.CoarseDropout(p=0.5, max_height=3, max_width=3, fill_value=0),
        # A.Cutout(p=0.5, max_h_size=3, max_w_size=3, fill_value=0),
        ToTensorV2(p=1.0),
    ], p=1.)

    test_transform = A.Compose([
        A.Normalize(mean=mean.tolist(), std=std.tolist(), max_pixel_value=1.0, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.)


    train_dataset = KeystrokesDataset(train_dataset_numpy, train_labels_numpy, train_transform)
    test_dataset = KeystrokesDataset(test_dataset_numpy, test_labels_numpy, test_transform)

    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

    train_dataloaders.append(train_dataloader)
    test_dataloaders.append(test_dataloader)

    return train_dataloaders, test_dataloaders

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters
num_epochs = 50
batch_size = 128
lr = 0.0002
beta1 = 0.5
beta2 = 0.999
nz = 100  # Size of generator input noise vector
ngf = 64  # Size of generator feature maps
ndf = 64  # Size of discriminator feature maps
nc = 5    # Number of channels in input images


class CustomDataset(Dataset):
    def __init__(self, data_path):
        self.data = np.load(data_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        sample = sample.transpose((1, 2 ,0))
        sample = torch.from_numpy(sample).float()
        # Normalize between -1 and 1
        sample = (sample - 0.5) / 0.5
        return sample
    
    
# Create the dataset and data loader
dataset = CustomDataset(all_files[0])


# Define the dataloader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Initialize the generator and discriminator
netG = Generator(nz, ngf, nc).to(device)
netD = Discriminator(ndf, nc).to(device)

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, beta2))
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, beta2))

# Train the generator and discriminator
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        print(data.shape)
        # Train the discriminator with real data
        netD.zero_grad()
        real_data = data[0].to(device)
        b_size = real_data.size(0)
        label = torch.full((b_size,), 1, device=device)
        output = netD(real_data).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # Train the discriminator with fake data
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake_data = netG(noise)
        label.fill_(0)
        output = netD(fake_data.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        # Train the generator
        netG.zero_grad()
        label.fill_(1)
        output = netD(fake_data).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        # Print the loss and save the generated images
        if i % 100 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))



torch.Size([110, 42, 42, 5])


RuntimeError: Given groups=1, weight of size [5, 64, 4, 4], expected input[1, 42, 42, 5] to have 64 channels, but got 42 channels instead

In [18]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define hyperparameters
lr = 0.0002
batch_size = 64
latent_dim = 100
num_epochs = 100

# Define the generator and discriminator
generator = Generator(latent_dim=latent_dim, input_shape=(5, 64, 64), hidden_dim=128).to(device)
discriminator = Discriminator(input_shape=(5, 64, 64), hidden_dim=128).to(device)

# Define loss function and optimizers
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        return torch.tensor(self.data[index], dtype=torch.float32)

    def __len__(self):
        return len(self.data)

# Load the dataset
# data = np.random.rand(1000, 5, 64, 64)  # example dataset
# dataset = CustomDataset(data)



###
data_extracted = extract_samples(0)
data_reshaped = data_extracted.reshape(len(data_extracted), 5, -1)
dataset = CustomDataset(data_reshaped)
###

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


# Define the training loop
for epoch in range(num_epochs):
    for i, imgs in enumerate(dataloader):
        # Adversarial ground truths
        valid = torch.ones((imgs.shape[0], 1), requires_grad=False, device=device)
        fake = torch.zeros((imgs.shape[0], 1), requires_grad=False, device=device)

        # Configure input
        real_imgs = imgs.to(device)

        # ------------------
        # Train Generator
        # ------------------

        optimizer_G.zero_grad()

        # Sample noise
        z = torch.randn(imgs.shape[0], latent_dim, requires_grad=False, device=device)

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ------------------
        # Train Discriminator
        # ------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

    # Print some statistics
    if epoch % 10 == 0:
        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, num_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

    # Save generated images
    # if epoch % 50 == 0:
    #     save_image(gen_imgs.data[:25], "images/%d.png" % epoch, nrow=5, normalize=True)


********************** 0
Current positive keystroke images Data shape is (110, 5, 42, 42)


RuntimeError: Given groups=1, weight of size [64, 5, 3, 3], expected input[1, 64, 5, 1764] to have 5 channels, but got 64 channels instead